
<h1 id="1.-데이터-로드">1. 데이터 로드<a class="anchor-link" href="#1.-데이터-로드">¶</a></h1>


In [ ]:

import pandas as pd

data = pd.read_csv("./train.csv")



In [ ]:

data.shape



In [ ]:

data.describe()



In [ ]:

#라벨과 피쳐데이터 분리
y = data['label']
data = data.drop(['label'], axis=1)



In [ ]:

#train data와 test data로 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=2019)



In [ ]:

#데이터의 컬럼 리스트
columns = list(data)



In [ ]:

#train set의 평균과 표준편차
X_mean = X_train.mean()
X_sd = X_train.std()

#표준편차가 0인경우 나눌 수 없으므로 보정하기
for i in range(len(X_sd)):
    if X_sd[i] == 0:
        X_sd[i] = 0.01

#train set 스케일링
from sklearn.preprocessing import scale
X_train = pd.DataFrame(scale(X_train))

X_train.describe()



In [ ]:

X_sd



In [ ]:

#test set 스케일링
for i,name in enumerate(columns):
    X_test[name] = (X_test[name] - X_mean[i]) / X_sd[i]

X_test.describe()




<h1 id="2.-원본-데이터로-모델적용">2. 원본 데이터로 모델적용<a class="anchor-link" href="#2.-원본-데이터로-모델적용">¶</a></h1>



<h3 id="1)-knn">1) knn<a class="anchor-link" href="#1)-knn">¶</a></h3>


In [ ]:

from sklearn.neighbors import KNeighborsClassifier
import datetime
bf_knn = datetime.datetime.now()
knn = KNeighborsClassifier().fit(X_train, y_train)
aft_knn = datetime.datetime.now()
print("차원축소 전 knn에 걸린 시간은 {} 입니다".format(aft_knn - bf_knn) )



In [ ]:

# #먼저 grid search로 최적의 이웃수를 찾아 knn을 하면 좋을거같아서 두가지 방법으로 해봤는데
# #너무 느려서 안됐어요ㅜㅜ.. 

# # knn Grid Search
# from sklearn.model_selection import cross_val_score
# # grid Search 구간 생성
# k_range = list(range(1, 10))
# # 정확도
# k_scores = []
# # 생성한 구간으로 grid search
# for k in k_range:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     # i에 따라 교차검증으로 정확도 출력
#     scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
#     # i번째 정확도를 추가
#     k_scores.append(scores.mean())
# print(k_scores)

# #grid search로 최적의 파라미터 찾기(교차검증=5)
# from sklearn.model_selection import GridSearchCV
# parameters = {'n_neighbors':[1,3,5,7,9]}
# clf = GridSearchCV(knn, parameters, cv=5)
# #데이터에 fit
# clf.fit(X_train,y_train)
# clf.best_params_
# clf.score(X_train,y_train)



In [ ]:

#학습한 모델로 test set 예측
knn_predict = knn.predict(X_test)



In [ ]:

#정확도 찍어보기
from sklearn.metrics import accuracy_score
knn_accuracy = accuracy_score(knn_predict, y_test)
print(knn_accuracy)




<h3 id="2)-Random-Forest">2) Random Forest<a class="anchor-link" href="#2)-Random-Forest">¶</a></h3>


In [ ]:

from sklearn.ensemble import RandomForestClassifier
bf_rfc = datetime.datetime.now()
rfc = RandomForestClassifier(random_state=42).fit(X_train,y_train)
aft_rfc = datetime.datetime.now()
print("차원축소 전 Random Forest에 걸린 시간은 {} 입니다".format(aft_rfc - bf_rfc) )



In [ ]:

# #마찬가지로 랜덤포레스트도 grid search를 적용할 수 있는데 시간이 오래걸려 생략했습니다!

# param_grid = { 
#     'n_estimators': [100,300,500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'criterion' :['gini', 'entropy']
# }
# CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
# CV_rfc.fit(X_train, y_train)
# CV_rfc.best_params_



In [ ]:

#학습한 모델로 test set예측
rfc_predict = rfc.predict(X_test)



In [ ]:

#정확도 찍어보기
rfc_accuracy = accuracy_score(rfc_predict, y_test)
print(rfc_accuracy)




<h1 id="3.-PCA로-차원축소하고-모델적용">3. PCA로 차원축소하고 모델적용<a class="anchor-link" href="#3.-PCA로-차원축소하고-모델적용">¶</a></h1>



<h3 id="1)PCA">1)PCA<a class="anchor-link" href="#1)PCA">¶</a></h3>


In [ ]:

from sklearn.decomposition import PCA
pca = PCA().fit(data)



In [ ]:

#모든 주성분은 784개
len(pca.explained_variance_)



In [ ]:

#screeplot 그려서 적당한 주성분 개수 찾아보기
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,5))
sing_vals = range(1,785)
eigvals = pca.explained_variance_
plt.plot(sing_vals, eigvals)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')
plt.show()



In [ ]:

#50개 이내에서 꺾이므로 50개로 다시 그려보기
fig = plt.figure(figsize=(8,5))
sing_vals = range(1,51)
eigvals = pca.explained_variance_[:50]
plt.plot(sing_vals, eigvals)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')
plt.show()



In [ ]:

#주성분의 개수를 10개로 설정하고, 데이터 transform하기
best_pca = PCA(n_components=10)
pca_features = pd.DataFrame(best_pca.fit_transform(data))



In [ ]:

#주성분을 10개로 할 때 설명력
sum(pca.explained_variance_[:10]) / sum(pca.explained_variance_)



In [ ]:

#transform한 데이터와 라벨을 붙여, pca결과 데이터 생성하기
pca_data = pd.concat([pca_features, y], axis=1)



In [ ]:

#pca_data의 컬럼 수는 11개
pca_data.shape



In [ ]:

pca_y = pca_data['label']
pca_X = pca_data.drop(['label'],axis=1)



In [ ]:

#train 과 test로 split
pca_X_train, pca_X_test, pca_y_train, pca_y_test = train_test_split(pca_X, pca_y, test_size=0.2, random_state=2019)




<h3 id="2)-knn">2) knn<a class="anchor-link" href="#2)-knn">¶</a></h3>


In [ ]:

#knn적용하기
pca_bf_knn = datetime.datetime.now()
pca_knn = KNeighborsClassifier().fit(pca_X_train, pca_y_train)
pca_aft_knn = datetime.datetime.now()
print("차원축소 후 knn에 걸린 시간은 {} 입니다".format(pca_aft_knn - pca_bf_knn) )



In [ ]:

#학습한 모델로 test set 예측
pca_knn_predict = pca_knn.predict(pca_X_test)



In [ ]:

#정확도 찍어보기
pca_knn_accuracy = accuracy_score(pca_knn_predict, pca_y_test)
print(pca_knn_accuracy)



In [ ]:

#pca이전 knn과 이후knn 정확도 비교하기
print("pca이전 knn의 정확도는 {}이고, 이후 정확도는 {}입니다".format(knn_accuracy, pca_knn_accuracy))



In [ ]:

#차원을 축소해서 예측의 정확도가 0.01정도 떨어졌지만, train data에 fit하는 속도와 test 데이터로 예측하는 속도는 많이 빨라졌다.




<h3 id="3)Random-Forest">3)Random Forest<a class="anchor-link" href="#3)Random-Forest">¶</a></h3>


In [ ]:

#랜덤포레스트 적용하기
pca_bf_rfc = datetime.datetime.now()
pca_rfc = RandomForestClassifier(random_state=42).fit(pca_X_train,pca_y_train)
pca_aft_rfc = datetime.datetime.now()
print("차원축소 후 Random Forest에 걸린 시간은 {} 입니다".format(pca_aft_rfc - pca_bf_rfc) )



In [ ]:

#학습한 모델로 test set예측
pca_rfc_predict = pca_rfc.predict(pca_X_test)



In [ ]:

#정확도 찍어보기
pca_rfc_accuracy = accuracy_score(pca_rfc_predict, pca_y_test)
print(pca_rfc_accuracy)



In [ ]:

#pca이전 randomforest과 이후randomforest 정확도 비교하기
print("pca이전 RandomForest의 정확도는 {}이고, 이후 정확도는 {}입니다".format(rfc_accuracy, pca_rfc_accuracy))



In [ ]:

#RandomForest의 정확도는 pca이후(knn보다 큰) 0.06정도 감소하였지만, 마찬가지로 속도가 빨라졌다.
#knn은 거리기반이기 때문에 차원의 영향이 더 크고, randomforest는 decision tree 기반이라 차원의 영향력이 더 적은것 같다.
#그래서 차원축소 전과 후 knn의 구동시간의 차이가 더 큰 것 같다.

